In [ ]:
!nvidia-smi

Fri May 14 00:54:19 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#!git clone https://github.com/seujung/KoBART-summarization /content/drive/MyDrive/kobart

import pandas as pd

In [ ]:
cd /content/drive/MyDrive/kobart

/content/drive/MyDrive/kobart


In [ ]:
"""
df = pd.read_csv("/content/drive/MyDrive/kobart/data/test.tsv", sep = '\t')
df.columns
df.head(1)
"""

'\ndf = pd.read_csv("/content/drive/MyDrive/kobart/data/test.tsv", sep = \'\t\')\ndf.columns\ndf.head(1)\n'

In [ ]:
!pip install gdown
!python download_binary.py

== Data existed.==


In [ ]:
#!cd KoBART-summarization/
!pip install -r requirements.txt
!pip install torchtext==0.8.0 

     |████████████████████████████████| 776.8MB 20kB/s 
     |████████████████████████████████| 1.9MB 50.0MB/s 
     |████████████████████████████████| 675kB 42.9MB/s 
     |████████████████████████████████| 7.4MB 26.1MB/s 
     |████████████████████████████████| 901kB 45.9MB/s 
     |████████████████████████████████| 3.3MB 39.1MB/s 
     |████████████████████████████████| 829kB 41.6MB/s 
     |████████████████████████████████| 645kB 49.1MB/s 
     |████████████████████████████████| 112kB 60.6MB/s 
     |████████████████████████████████| 81kB 10.6MB/s 
     |████████████████████████████████| 112kB 61.7MB/s 
     |████████████████████████████████| 163kB 50.6MB/s 
     |████████████████████████████████| 4.2MB 44.9MB/s 
     |████████████████████████████████| 71kB 9.8MB/s 
     |████████████████████████████████| 122kB 57.4MB/s 
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=5bc38281825819974b19e2622645890d968855726ef71f274b0d5d13f7d82e91
  Stored i

     |████████████████████████████████| 7.0MB 12.6MB/s 
  Found existing installation: torchtext 0.9.1
    Uninstalling torchtext-0.9.1:
      Successfully uninstalled torchtext-0.9.1


In [ ]:
!bash install_kobart.sh

  Cloning https://github.com/SKT-AI/KoBART to /tmp/pip-install-ofgbtxf2/kobart
  Running command git clone -q https://github.com/SKT-AI/KoBART /tmp/pip-install-ofgbtxf2/kobart
  Created wheel for kobart: filename=kobart-0.4-cp37-none-any.whl size=8502 sha256=4cde3dd220ab49c7688966b9009b41e3471bf512258a348e707ea190bc7eef10
  Stored in directory: /tmp/pip-ephem-wheel-cache-x8g713mn/wheels/4a/90/60/b0692d915b1dcfa5556fcca48a1422e031118bbae70977d2ae
Successfully built kobart


## Train

In [ ]:
!python train.py  --gradient_clip_val 1.0 --max_epochs 40 --default_root_dir logs  --gpus 1 --batch_size 4 --num_workers 4 --resume_from_checkpoint '/content/drive/MyDrive/kobart/logs/kobart_summary-last.ckpt'

INFO:root:Namespace(accelerator=None, accumulate_grad_batches=1, amp_backend='native', amp_level='O2', auto_lr_find=False, auto_scale_batch_size=False, auto_select_gpus=False, automatic_optimization=None, batch_size=4, benchmark=False, check_val_every_n_epoch=1, checkpoint_callback=True, checkpoint_path=None, default_root_dir='logs', deterministic=False, distributed_backend=None, enable_pl_optimizer=True, fast_dev_run=False, flush_logs_every_n_steps=100, gpus=1, gradient_clip_val=1.0, limit_test_batches=1.0, limit_train_batches=1.0, limit_val_batches=1.0, log_every_n_steps=50, log_gpu_memory=None, logger=True, lr=3e-05, max_epochs=40, max_len=512, max_steps=None, min_epochs=1, min_steps=None, model_path=None, move_metrics_to_cpu=False, num_nodes=1, num_processes=1, num_sanity_val_steps=2, num_workers=4, overfit_batches=0.0, plugins=None, precision=32, prepare_data_per_node=True, process_position=0, profiler=None, progress_bar_refresh_rate=1, reload_dataloaders_every_epoch=False, replac

## binary 파일로 변환

In [ ]:
!python get_model_binary.py --hparams '/content/drive/MyDrive/kobart/logs/tb_logs/default/version_1/hparams.yaml' --model_binary '/content/drive/MyDrive/kobart/logs/kobart_summary-model_chp/epoch=02-val_loss=1.339.ckpt'
  

get_model_binary.py:13: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  hparams = yaml.load(f)
using cached model
using cached model


## Test

In [ ]:
import torch
from kobart import get_kobart_tokenizer
from transformers.models.bart import BartForConditionalGeneration

def load_model():
    model = BartForConditionalGeneration.from_pretrained('./kobart_summary')
    # tokenizer = get_kobart_tokenizer()
    return model

model = load_model()
tokenizer = get_kobart_tokenizer()

text = input()

if text:
    text = text.replace('\n', '')
    print("## KoBART 요약 결과")
    input_ids = tokenizer.encode(text)
    input_ids = torch.tensor(input_ids)
    input_ids = input_ids.unsqueeze(0)
    output = model.generate(input_ids, eos_token_id=1, max_length=512, num_beams=5)
    output = tokenizer.decode(output[0], skip_special_tokens=True)
    print(output)

using cached model
부동산 투자로 억대의 돈을 번 상도는 최근 N×N 크기의 땅을 구매했다. 상도는 손쉬운 땅 관리를 위해 땅을 1×1 크기의 칸으로 나누어 놓았다. 각각의 칸은 (r, c)로 나타내며, r은 가장 위에서부터 떨어진 칸의 개수, c는 가장 왼쪽으로부터 떨어진 칸의 개수이다. r과 c는 1부터 시작한다.상도는 전자통신공학과 출신답게 땅의 양분을 조사하는 로봇 S2D2를 만들었다. S2D2는 1×1 크기의 칸에 들어있는 양분을 조사해 상도에게 전송하고, 모든 칸에 대해서 조사를 한다. 가장 처음에 양분은 모든 칸에 5만큼 들어있다.매일 매일 넓은 땅을 보면서 뿌듯한 하루를 보내고 있던 어느 날 이런 생각이 들었다.나무 재테크를 하자!나무 재테크란 작은 묘목을 구매해 어느정도 키운 후 팔아서 수익을 얻는 재테크이다. 상도는 나무 재테크로 더 큰 돈을 벌기 위해 M개의 나무를 구매해 땅에 심었다. 같은 1×1 크기의 칸에 여러 개의 나무가 심어져 있을 수도 있다.이 나무는 사계절을 보내며, 아래와 같은 과정을 반복한다.봄에는 나무가 자신의 나이만큼 양분을 먹고, 나이가 1 증가한다. 각각의 나무는 나무가 있는 1×1 크기의 칸에 있는 양분만 먹을 수 있다. 하나의 칸에 여러 개의 나무가 있다면, 나이가 어린 나무부터 양분을 먹는다. 만약, 땅에 양분이 부족해 자신의 나이만큼 양분을 먹을 수 없는 나무는 양분을 먹지 못하고 즉시 죽는다.여름에는 봄에 죽은 나무가 양분으로 변하게 된다. 각각의 죽은 나무마다 나이를 2로 나눈 값이 나무가 있던 칸에 양분으로 추가된다. 소수점 아래는 버린다.가을에는 나무가 번식한다. 번식하는 나무는 나이가 5의 배수이어야 하며, 인접한 8개의 칸에 나이가 1인 나무가 생긴다. 어떤 칸 (r, c)와 인접한 칸은 (r-1, c-1), (r-1, c), (r-1, c+1), (r, c-1), (r, c+1), (r+1, c-1), (r+1, c), (r+1, c+1) 이다. 상도의 땅을 벗어나는 칸에는 나무가 생